# House Prices - Kaggle competition

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview

Quels sont les critères de succès du présent projet ? 
1. précision des prédictions
...

We load the train data. The PassengerId column is used as the index of the dataframe

In [3]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import optuna
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error

# Load the data
train = pd.read_csv("Data/train.csv", index_col="Id")
test = pd.read_csv("Data/test.csv", index_col="Id")
train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

## Preprocessing Pipeline

We identified null values in all columns. We will clean these by type.

In [5]:
train.isna().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 80, dtype: int64

In [8]:
# pip install scikit-learn

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

#mixed regression à faire

# Step 1: Define transformers for different column types
numerical_cols = [
    'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
    'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
    '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
    'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
    'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
    'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'
]
categorical_cols = [
    'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
    'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
    'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
    'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
    'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
    'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
    'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'
]

# Define transformers for the preprocessing step
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Step 2: Create a ColumnTransformer that applies the transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop'
)

# Step 3: Assemble the preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

# Prepare features and target
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']

# Fit and transform the DataFrame
X_prepared = preprocessing_pipeline.fit_transform(X)

preprocessing_pipeline



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
                                                   'GrLivArea', 'BsmtFullBa...
                                                   'Street', 'Alley',
                                                   'LotShape', 'LandContour',
                                                   'Utilities', 'LotConfig',
                                                   'LandSlope', 'Neighborhood',
                                                   'Condition1', 'Condition2',
                                                   'BldgType', 'HouseStyle',
                                                   'RoofStyle', 'RoofMatl',
                                                   'Exterior1st', 'Exterior2nd',
                                                   'MasVnrType', 'ExterQual',
                                                   'ExterCond', 'Foundation',
                                                   'BsmtQual', 'BsmtCond',
                                                   'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir', ...])]))])

In [7]:
from scipy import sparse  # For handling sparse matrix outputs

# Handling the conversion from sparse matrix if necessary
if sparse.issparse(X_prepared):
    X_prepared = X_prepared.toarray()

# Fetch feature names for categorical columns
# First, we need to correctly access the feature names from the OneHotEncoder
    onehot_encoder = preprocessing_pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['encoder']
    onehot_features = onehot_encoder.get_feature_names_out(categorical_cols)
    feature_names = numerical_cols + list(onehot_features)

# Ensure the number of columns matches

    X_preprocessed = pd.DataFrame(X_prepared, columns=feature_names, index=train.index)
X_preprocessed.head()


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,-0.229372,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.511418,0.575425,-0.288653,-0.944591,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.451936,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574410,1.171992,-0.288653,-0.641228,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.093110,0.073480,0.651479,-0.517200,0.984752,0.830215,0.323060,0.092907,-0.288653,-0.301643,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.456474,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574410,-0.499274,-0.288653,-0.061670,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.633618,0.375148,1.374795,-0.517200,0.951632,0.733308,1.364570,0.463568,-0.288653,-0.174865,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Hyperparameter Tuning of a Decision Tree Regressor

We extend the pipeline with a decision tree regressor (DecisionTreeRegressor) to predict house sale prices.

In [15]:
import numpy as np
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from scipy import sparse
from sklearn.model_selection import KFold

# Custom function to calculate RMSLE
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

def objective(trial):
    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True)
    }

    model = xgb.XGBRegressor(**xgb_params, random_state=42, objective='reg:squarederror')
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmsle_scores = []
    
    for train_index, valid_index in kf.split(X_preprocessed):
        X_train, X_valid = X_preprocessed.iloc[train_index], X_preprocessed.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        model.fit(X_train, np.log1p(y_train), early_stopping_rounds=50, eval_set=[(X_valid, np.log1p(y_valid))], verbose=False)
        preds = model.predict(X_valid)
        score = rmsle(np.log1p(y_valid), preds)
        rmsle_scores.append(score)
    
    return np.mean(rmsle_scores)

# Use Optuna with TPE sampler for Bayesian optimization
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=200)  # Increased number of trials for better hyperparameter tuning

print("Best hyperparameters =", study.best_trial.params)

# Fit the final model with the best hyperparameters
final_model = xgb.XGBRegressor(**study.best_trial.params, random_state=42, objective='reg:squarederror')
final_model.fit(X_preprocessed, np.log1p(y))  # Use all training data



[I 2024-05-21 10:54:24,047] A new study created in memory with name: no-name-36715ea6-8bb6-48f2-ab3b-2d3e4bae2af4
c:\Users\blerdon\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\blerdon\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\blerdon\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\blerdon\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopp

Best hyperparameters = {'n_estimators': 1881, 'max_depth': 7, 'learning_rate': 0.011060961787479538, 'subsample': 0.5093901211150806, 'colsample_bytree': 0.314091116841493, 'min_child_weight': 10, 'reg_alpha': 0.00013176446413887263, 'reg_lambda': 7.925911480640328e-07, 'gamma': 1.2352515697597087e-05}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.314091116841493, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=1.2352515697597087e-05,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.011060961787479538,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=10, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1881, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

RMSLE: Un RMSLE de 0.1327 est relativement faible, ce qui est bon, car cela signifie que les erreurs entre les valeurs prédites et les valeurs réelles ne sont pas très élevées sur une échelle logarithmique. Ce métrique pénalise moins les erreurs sur les grandes valeurs absolues, ce qui est utile dans le cas de données de prix où l'échelle peut varier largement.

In [16]:
from optuna.visualization import plot_optimization_history
plot_optimization_history(study)

### Using MLFlow for evaluation

The best model is saved in MLFlow for later use. First we need to start a local tracking server. After installing mlflow, execute the following command in the terminal:

mlflow server --host 127.0.0.1 --port 8080

In [10]:
# pip install mlflow
# pip install setuptools

In [13]:
import mlflow
from mlflow.models import infer_signature

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")
mlflow.set_experiment("Spaceship_Titanic")

2024/03/18 23:58:35 INFO mlflow.tracking.fluent: Experiment with name 'Spaceship_Titanic' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/927528599907738554', creation_time=1710802715345, experiment_id='927528599907738554', last_update_time=1710802715345, lifecycle_stage='active', name='Spaceship_Titanic', tags={}>

## Random Forest

We extend the pipeline with a RandomForest classifier to predict the Transported variable. 

In [19]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

model = RandomForestClassifier(random_state=42, n_jobs=-1)

def objective(trial):
    
    params = {
        # trial parameters to optimize for RandomForestClassifier
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 50, log=True),
        'min_samples_split': trial.suggest_float('min_samples_split', 1e-6, 1e-3, log=True),
        'min_samples_leaf': trial.suggest_float('min_samples_leaf', 1e-6, 1e-3, log=True),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0)
    }

    model.set_params(**params)

    cv_score = cross_val_score(model, X_preprocessed, y, cv=5, scoring='accuracy', n_jobs=-1).mean()

    return cv_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("--------------------------------------")
print("best_params =", study.best_params, "with cross_validation_score =", study.best_value)

[I 2024-03-19 00:20:24,105] A new study created in memory with name: no-name-2d95a428-b5ff-4550-bd76-43fe89a2dee3
[I 2024-03-19 00:20:25,276] Trial 0 finished with value: 0.7423237697948174 and parameters: {'n_estimators': 589, 'max_depth': 3, 'min_samples_split': 6.066134126699176e-06, 'min_samples_leaf': 1.994219971772114e-06, 'max_features': 0.16839304211453318}. Best is trial 0 with value: 0.7423237697948174.
[I 2024-03-19 00:20:26,225] Trial 1 finished with value: 0.7907537167704148 and parameters: {'n_estimators': 203, 'max_depth': 21, 'min_samples_split': 0.000856834446460495, 'min_samples_leaf': 6.459669547653005e-06, 'max_features': 0.17866570136904392}. Best is trial 1 with value: 0.7907537167704148.
[I 2024-03-19 00:20:26,912] Trial 2 finished with value: 0.7802870054149343 and parameters: {'n_estimators': 167, 'max_depth': 4, 'min_samples_split': 4.065099596768056e-05, 'min_samples_leaf': 0.00021869235326431172, 'max_features': 0.6831354417525116}. Best is trial 1 with valu

--------------------------------------
best_params = {'n_estimators': 908, 'max_depth': 9, 'min_samples_split': 1.9286857153060706e-05, 'min_samples_leaf': 7.083051075025364e-05, 'max_features': 0.5803462002221914} with cross_validation_score = 0.7992658770466473


In [20]:
train = pd.read_csv("Data/train.csv", index_col= "PassengerId")

X = train.drop(['Transported'], axis = 1)
y = train['Transported']

# Start an MLflow run
with mlflow.start_run():
    # Fit the model with the best hyperparameters from the study
    model = RandomForestClassifier(random_state=42, n_jobs=-1)
    model.set_params(**study.best_params)

    model_pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('scaler', StandardScaler()),
            ('classifier', model)
        ])  

    model_pipeline.fit(X, y)
    
    # Log the hyperparameters
    mlflow.log_params(study.best_params)

    # Log the loss metric
    mlflow.log_metric("accuracy", study.best_value)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Random Forest Classifier")

    # Infer the model signature
    signature = infer_signature(X, model_pipeline.predict(X))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model_pipeline,
        signature=signature,
        input_example=X,
        registered_model_name="random_forest_model",
        artifact_path="random_forest_model"
    )

Successfully registered model 'random_forest_model'.
2024/03/19 00:24:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model, version 1
Created version '1' of model 'random_forest_model'.


#Evaluation on Kaggle


We will now publish on kaggle our tree with the most optimal depth, min_samples_leaf and min_samples_split.



In [17]:
# Prepare and predict the test data
X_test_prepared = preprocessing_pipeline.transform(test)
if sparse.issparse(X_test_prepared):
    X_test_prepared = X_test_prepared.toarray()

preds_test = np.expm1(final_model.predict(X_test_prepared))

# Output predictions
output = pd.DataFrame({'Id': test.index, 'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
